# Dimension Reduction of correlated features group via PCA

In [ ]:
count_feature_subset = 

In [ ]:
class PipeLine:
    def __init__(self, steps = None) -> None:
        self.steps = [] if steps is None else steps
        
    def add_step(self, f, *args, **kwargs):
        self.steps.append((f, args, kwargs))
    
    def run_all(self, df):
        for step, args, kwargs in self.steps:
            df = step(df, *args, **kwargs)
        return df
preprocess_pipes = PipeLine()

In [ ]:
from sklearn.decomposition import PCA
def pca_reduce_fit_transform(df, cols, nm):
    non_na_mask = df[cols].isna().all(axis=1)
    non_na_idx = non_na_mask[~non_na_mask].index

    pca = PCA(1).fit(df[cols].dropna())
    df.loc[non_na_idx, nm] = pca.transform(df.loc[non_na_idx, cols])
    
    return df, pca

def pca_reduce_fit(df, cols):
    non_na_mask = df[cols].isna().all(axis=1)
    non_na_idx = non_na_mask[~non_na_mask].index

    pca = PCA(1).fit(df[cols].dropna())
    
    return pca

def pca_reduce_transform(df, pca, nm):
    cols = pca.feature_names_in_
    non_na_mask = df[cols].isna().all(axis=1)
    non_na_idx = non_na_mask[~non_na_mask].index
    
    df.loc[non_na_idx, nm] = pca.transform(df.loc[non_na_idx, cols])
    
    return df, pca

In [ ]:
from itertools import count
red_grps = {}

for p, grps in enumerate(vesta_subset_pages[:2]):
    for cols in grps:
        c_it = count()
        if len(cols) <= 2:
            continue
        nm = f'PCA_{p}_{next(c_it)}'
        transaction_df, pca = pca_reduce_transform(transaction_df, cols, nm)
        red_grps[nm] = pca

In [ ]:
{k:len(pca.feature_names_in_) for k, pca in red_grps.items()}

In [ ]:
cols = ['PCA_7_0'] + list(red_grps['PCA_7_0'].feature_names_in_)
cor = transaction_df[cols].corr()
sns.heatmap(cor,cmap='Blues');